# Projet : Consommation et production électrique en France

## Etude des habitudes de consommations 

Dans cette partie vous choisirez le dataframe (national ou régional) qui vous paraît le plus adapté en fonction des questions. 

&#x1F4A5; **To Do**

    - A quelle heure de la journée consomme-t-on en moyenne le moins ? 

    - Afficher les moyennes des consommations selon le jour de la semaine. Est-ce logique ? 

    - Tracer ces mêmes courbes, en ne sélectionnant que les jours de semaine, puis que les jours de week end. Quelles différences de comportement peut-on observer ?
    
    - Tracer les consommations horaires (par heure) moyenne en décembre et en juin. Peut-on observer les différences d'ensoleillement entre les régions ?
    
    - Afficher les moyennes des consommations régionales pour chaque année, avec et sans standardisation. Quelles informations visualisez vous ?


**Aide**

- un `DatetimeIndex` possède de nombreux attributs et méthodes utiles: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html
- Regarder la doc des méthodes `groupby`, `unstack`, `pivot`...
- Si besoin, utiliser la méthode `seaborn.heatmap` pour visualiser vos résultats (`cmap="YlGnBu"`)

#### moyennes des consommations régionales pour chaque année, avec et sans standardisation

In [ ]:
# Laisser le premier code par année en exemple pour s'approprier le groupby ?

In [ ]:
# sans standardisation
groupbyyear = df_regional_swaped['Consommation'].groupby(df_regional.index.year)
groupbyyear.mean().plot(figsize=(15, 4))

In [ ]:
groupbyyear.head()

In [ ]:
# avec standardisation
groupbyyear.mean().transform(lambda x: (x - x.mean()) / x.std()).plot(figsize=(15, 4))

La standardisation ne change pas le comportement des données, change l'échelle par contre : on voit mieux ce qu'il se passe dans les données

In [ ]:
# avec standardisation
import seaborn as sns
sns.heatmap(groupbyyear.mean().transform(lambda x: (x - x.mean()) / x.std(), axis=0), cmap="YlGnBu")

#### moyenne de consommation selon le jour de la semaine

In [ ]:
# sans standardisation
groupbyweek = df_regional_swaped['Consommation'].groupby(df_regional.index.dayofweek).mean()
groupbyweek.plot(figsize=(15, 4))

In [ ]:
# avec standardisation
groupbyweek.transform(lambda x: (x - x.mean()) / x.std(), axis=0).plot(figsize=(15, 4))

#### consommations décembre vs juin

In [ ]:
conso=df_regional_swaped.Consommation
conso_june=conso[conso.index.month==6]
conso_dec=conso[conso.index.month==12]

In [ ]:
conso_june.head()

In [ ]:
conso_dec.groupby(conso_dec.index.time).mean()

In [ ]:
# par heure
conso=df_regional_swaped.Consommation
conso_june=conso[conso.index.month==6]
conso_dec=conso[conso.index.month==12]
conso_june.groupby(conso_june.index.time).mean().plot(figsize=(12, 4))
conso_dec.groupby(conso_dec.index.time).mean().plot(figsize=(12, 4))

In [ ]:
# avec standardisation
conso_june.groupby(conso_june.index.time).mean().transform(lambda x: (x - x.mean()) / x.std(), axis=0).plot(figsize=(12, 4))
conso_dec.groupby(conso_dec.index.time).mean().transform(lambda x: (x - x.mean()) / x.std(), axis=0).plot(figsize=(12, 4))

#### consommation semaine vs week end

In [ ]:
# la semaine
df_dec = conso[(conso.index.month == 12) & (conso.index.dayofweek < 5)]
groupbyhour = df_dec.groupby(df_dec.index.hour)
groupbyhour.mean().transform(lambda x: (x - x.mean()) / x.std()).plot(figsize=(12, 4))

In [ ]:
# le week-end
df_dec = conso[(conso.index.month == 12) & (conso.index.dayofweek > 4)]
groupbyhour = df_dec.groupby(df_dec.index.hour)
groupbyhour.mean().transform(lambda x: (x - x.mean()) / x.std()).plot(figsize=(12, 4))

## Relation entre les features / variables

## Liens productions / consommation

**TODO**
        
- Afficher les correlations entre toutes les productions et la consommation
- Quelles éléments sont corrélés parce que liés ? Quels éléments sont corrélés mais n'ont à priori pas de lien de cause à effet ?
- Quel indice montre que la consommation 'cause' la production avec du gaz ?
- Quels types de productions permettent de "suivre" la consommation ? Lesquels ne le permettent pas du tout (productions "fatales")
- Choisissez une ou deux régions et observez ces mêmes corrélations.

Le nucléaire produit beaucoup, mais est peu pilotable. <br/>
En revanche le gaz est allumé lorsqu'il y a beaucoup de demande : la demande cause la production de gaz.

### Au niveau national

#### Afficher les correlations entre toutes les productions et la consommation

In [ ]:
sns.heatmap(df_national[prod_+['Consommation']].corr(), cmap="RdBu_r", vmin=-1, vmax=1, linewidths=0.1)

Nucléaire et gaz corrélés avec la consommation : on pourrait penser que plus on consomme plus on a besoin de ces sources de production

#### Quels éléments sont corrélés parce que liés ? Quels éléments sont corrélés mais n'ont à priori pas de lien de cause à effet ?

Les corrélations peuvent être dues à une relation de causalité, par exemple, une hausse rapide de la consommation va entrainer une hausse de la production avec du gaz. Ainsi on peut noter que la production de gaz suit une hausse de la consommation. Ce genre d'intuition est à l'origine de la notion de [causalité de Granger](https://en.wikipedia.org/wiki/Granger_causality)

Gaz et nucléaire semblent liés : mais uniquement parce qu'ils sont liés à la consommation

#### Quel indice montre que la consommation 'cause' la production avec du gaz?

In [ ]:
comp = df_national[['Gaz', 'Consommation']].dropna()
comp =comp/comp.max()
comp.iloc[:1000].plot(figsize=(12, 4))

Ci dessous on vérifie que la production de gaz suit en moyenne la hausse de la consommation :<br/>
An idea from: https://stackoverflow.com/questions/33171413/cross-correlation-time-lag-correlation-with-pandas

In [ ]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

In [ ]:
import matplotlib.pyplot as plt
lags = range(-3, 3)
lagged_corr = [crosscorr(comp['Gaz'], comp['Consommation'], lag=lag) for lag in lags]
plt.plot(lags, lagged_corr);plt.show()

#### Quels types de productions permettent de "suivre" la consommation ? Lesquels ne le permettent pas du tout (productions "fatales")

De la même manière une hausse de la consommation va entrainer une hausse de la production nucléaire, mais comme des centrales nucléaires ne peuvent pas être allumées rapidement, cette hausse se fait à l'échelle de l'année :

In [ ]:
comp = df_national[['Nucléaire', 'Consommation']].dropna()
comp =comp/comp.max()
comp.iloc[:1000].plot(figsize=(12, 4))

In [ ]:
comp = df_national[['Nucléaire', 'Consommation']].dropna()
comp =comp/comp.max()
comp.resample('1D').mean().plot(figsize=(12, 4))

In [ ]:
import matplotlib.pyplot as plt
lags = range(-3, 3)
comp = df_national[['Nucléaire', 'Consommation']].dropna()
comp =comp/comp.max()
lagged_corr = [crosscorr(comp['Nucléaire'], comp['Consommation'], lag=lag) for lag in lags]
plt.plot(lags, lagged_corr);plt.show()

En revanche le solaire ou l'éolien sont peu pilotables :

In [ ]:
comp = df_national[['Solaire', 'Eolien', 'Consommation']].dropna()
comp =comp/comp.max()
comp.resample('1D').mean().plot(figsize=(12, 4))

In [ ]:
comp = df_national[['Solaire', 'Eolien', 'Consommation']].dropna()
comp =comp/comp.max()
comp.iloc[:500].plot(figsize=(12, 4))

### Au niveau régional

Même démarche

In [ ]:
sns.heatmap(df_regional['Auvergne-Rh├┤ne-Alpes'].corr(), cmap="RdBu_r", vmin=-1, vmax=1, linewidths=0.1)

In [ ]:
sns.heatmap(df_regional['Ile-de-France'].corr(), cmap="RdBu_r", vmin=-1, vmax=1, linewidths=0.1)

In [ ]:
sns.heatmap(df_regional['PACA'].corr(), cmap="RdBu_r", vmin=-1, vmax=1, linewidths=0.1)

## BONUS : Prix de l'électricité

On s'intéresse maintenant aux données dans le dossier "data entsoe".<br/>
https://www.opendatadsl.com/docs/company/ENTSOE#datasets

**TODO**

- Charger les prix de marché de l'électricité  (en France et en Allemagne)  <br/>
- Chercher les liens (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html) entre : 
    - Le prix de l'électricité et la consommation nationale ?
    - Le prix de l'électricité et les sources de production ? 
    - Quelles sources semblent associées aux prix élevés ? aux prix faibles ?
            
**Aide:**

- Attention aux timezone ! Les prix sont en UTC. Les méthodes **tz_localize('UTC') et tz_convert('Europe/Paris')** pourraient vous aider ...
- On ne peut pas comparer timeserie "localisée" avec une autre qui ne l'est pas ("naive")
- Pour associer le dataframe de conso/prod à une timezone : les changements d'heures ont mal été gérés (cherchez à comprendre le problème). Utliser **"ambiguous = 'NaT', nonexistent='NaT'"** pour le contourner.
- Y a-t-il des **outliers** ?

#### Charger les prix de marché de l'électricité (en France et en Allemagne)

In [ ]:
def read_data_entsoe(file):
    df = pd.read_csv(file)
    df['start'], _ = df['MTU (UTC)'].str.split(' - ', 1).str
    df.set_index('start', inplace=True)
    df.index = pd.to_datetime(df.index, yearfirst=True)
    df.drop(columns=['MTU (UTC)'], inplace=True)
    return df

In [ ]:
DATA_FILE = './data/data entsoe/France/'

def get_prices(folder, col='fr'):
    df_prices = []
    for f in listdir(folder):
        df = read_data_entsoe(folder + f)
        df.columns = [col]
        df_prices.append(df)
    df_prices= pd.concat(df_prices, axis=0)
    return df_prices

df_prices = pd.concat((get_prices('./data/data entsoe/Electricity_price/France/', 'fr'),
                         get_prices('./data/data entsoe/Electricity_price/Germany/', 'ger')), 1)


<details><summary>Code import données électricité</summary><br>
def read_data_entsoe(file):<br/>
    df = pd.read_csv(file)<br/>
    df['start'], _ = df['MTU (UTC)'].str.split(' - ', 1).str<br/>
    df.set_index('start', inplace=True)<br/>
    df.index = pd.to_datetime(df.index, yearfirst=True)<br/>
    df.drop(columns=['MTU (UTC)'], inplace=True)<br/>
    return df<br/>
    
DATA_FILE = './data/data entsoe/France/'<br/>

def get_prices(folder, col='fr'):<br/>
    df_prices = []<br/>
    for f in listdir(folder):<br/>
        df = read_data_entsoe(folder + f)<br/>
        df.columns = [col]<br/>
        df_prices.append(df)<br/>
    df_prices= pd.concat(df_prices, axis=0)<br/>
    return df_prices<br/>

df_prices = pd.concat((get_prices('./data/data entsoe/Electricity_price/France/', 'fr'),<br/>
                         get_prices('./data/data entsoe/Electricity_price/Germany/', 'ger')), 1)<br/>
</details>

In [ ]:
df_prices.describe()

In [ ]:
import numpy as np
df_prices.loc[df_prices.fr > 500, 'fr'] = np.nan # remplacement par nan des valeurs aberrantes
df_prices.describe()

In [ ]:
# assign time zone and convert to european time
df_prices = df_prices.tz_localize('UTC').tz_convert('Europe/Paris').dropna()
df_national = df_national.tz_localize('Europe/Paris', ambiguous = 'NaT', nonexistent='NaT').dropna()
# ambiguous = 'NaT', nonexistent='NaT' à cause des chgt d'heures ...

#### Recherche de liens

On a une faible corrélation positive entre consommation et prix :

In [ ]:
df_national['Consommation'].corr(df_prices.fr)

In [ ]:
# standardisation
df_price_std = ((df_prices-df_prices.mean())/df_prices.std()).dropna()
df_std = ((df_national-df_national.mean())/df_national.std())

In [ ]:
df_price_std.loc['2017-02'].dropna().plot()
df_std['Consommation'].loc['2017-02'].dropna().plot(figsize=(12, 4))

In [ ]:
common_ts = set(df_national[prod_].index).intersection(df_prices.index)
corr_ =pd.concat((df_national[prod_].loc[common_ts], df_prices.loc[common_ts]),1).corr()
sns.heatmap(corr_)

L'éolien fait baisser les prix, en particulier en Allemagne.

Le charbon et le gaz sont corrélés à un augmentation des prix (sources flexibles donc allumées quand il y a un manque).

In [ ]:
corr_[['fr', 'ger']]

On peut aussi regarder les productions associées aux prix élevés et faibles :

In [ ]:
index_price = df_price_std[df_price_std.fr < -2].index
print(len(index_price))
common_index = df_std[prod_].index.intersection(index_price)
df_std[prod_].loc[common_index].mean().sort_values(ascending=False)

In [ ]:
index_price = df_price_std[df_price_std.fr >2].index
print(len(index_price))
common_index = df_std[prod_].index.intersection(index_price)
df_std[prod_].loc[common_index].mean().sort_values(ascending=False)

## BONUS : Emissions de CO2

**TODO**

- Quel lien entre le taux de C02 et les différentes sources de productions ?
- Quelles sources semblent les plus émettrices ?

In [ ]:
# df_national_simpl.corr()['Taux de Co2'].sort_values(ascending=False)
df_national[prod_].corrwith(df_national['Taux de Co2']).sort_values(ascending=False)

Le charbon et le gaz sont les plus émetteurs de CO2

## BONUS : Caractéristation des échanges avec les frontières
**TODO**

- A quel moment les échanges ont-il principalement lieu ?
- Quel est le prix payé par la France lorsqu'elle importe de l'électricité ? 
- Quel est le prix reçu lors des exportations ?


` >0 = importation, <0 = exportation`

#### A quel moment les échanges ont-il principalement lieu ?

In [ ]:
Ech_comm = ['Ech. comm. Angleterre', 'Ech. comm. Espagne', 'Ech. comm. Italie', 
            'Ech. comm. Suisse', 'Ech. comm. Allemagne-Belgique']

In [ ]:
common_index = df_national[Ech_comm].dropna().index.intersection(df_price_std.index)
df_temp = pd.concat([df_national.loc[common_index, Ech_comm], df_price_std.loc[common_index]], axis=1)

In [ ]:
sns.heatmap(df_temp.corr(), cmap="RdBu_r", vmin=-1, vmax=1, linewidths=0.1)

In [ ]:
df_std[Ech_comm].groupby(df_national.index.day).mean().plot(figsize=(12, 4))

In [ ]:
df_std[Ech_comm].groupby(df_national.index.dayofweek).mean().plot(figsize=(12, 4))

#### Quel est le prix payé par la France lorsqu'elle importe / exporte de l'électricité ?

In [ ]:
# importations (france <= allemagne)
index_ech = df_national[df_national['Ech. comm. Allemagne-Belgique'] > 0].index
# prix_fr = round(df_price_std.loc[df_price_std.index.intersection(index_ech)].fr.mean(), 3)
# prix_ger = round(df_price_std.loc[df_price_std.index.intersection(index_ech)].ger.mean(), 3)
prix_fr = round(df_prices.loc[df_prices.index.intersection(index_ech)].fr.mean(), 3)
prix_ger = round(df_prices.loc[df_prices.index.intersection(index_ech)].ger.mean(), 3)
print('prix moy imp fr =', prix_fr, '; prix moy allemagne = ', prix_ger)

# exportations (france => allemagne)
index_ech = df_national[df_national['Ech. comm. Allemagne-Belgique'] < 0].index
# prix_fr = round(df_price_std.loc[df_price_std.index.intersection(index_ech)].fr.mean(), 3)
# prix_ger = round(df_price_std.loc[df_price_std.index.intersection(index_ech)].ger.mean(), 3)
prix_fr = round(df_prices.loc[df_prices.index.intersection(index_ech)].fr.mean(), 3)
prix_ger = round(df_prices.loc[df_prices.index.intersection(index_ech)].ger.mean(), 3)
print('prix moy exp fr =', prix_fr, '; prix moy allemagne = ', prix_ger)

In [ ]:
for pays in Ech_comm:
    index_exp = df_national[df_national[pays] < 0].index
    prix_exp = round(df_prices.loc[df_prices.index.intersection(index_exp)].fr.mean(), 3)
    index_imp = df_national[df_national[pays] > 0].index
    prix_imp = round(df_prices.loc[df_prices.index.intersection(index_imp)].fr.mean(), 3)
    print(pays, ': prix moy exp =', prix_exp, '(', str(len(index_exp)), 'h) ; prix moy imp =', 
          prix_imp, '(', str(len(index_imp)), 'h)')

In [ ]:
sns.heatmap(pd.concat([df_national[prod_], df_national[Ech_comm]], axis=1).corr(),
            cmap="RdBu_r", vmin=-1, vmax=1, linewidths=0.1)